In [ ]:
from srf2.meta.psf_meta import *
from srf2.meta.image_meta import *
from srf2.data.image import *
import scipy.optimize as opt
import numpy as np

_sqrt_pi = np.sqrt(np.pi)

from tqdm import tqdm_notebook
from matplotlib import pyplot as plt

filename_prefix1 = '/home/bill52547/Workspace/SRF_new_start/run/psf_2m/psf_img_xy_'
filename_prefix2 = '/home/bill52547/Workspace/SRF_new_start/run/psf_2m/psf_img_z_'
# image = np.load(filename)
meta = PsfMeta3d()
image_meta = Image_meta_3d((201, 201, 540), (0, 0, 1.71), (687.42, 687.42, 1846.8))

plt.figure(figsize = (30, 100))
for i in tqdm_notebook(range(100)):
    filename = filename_prefix1 + str(i) + '.npy'
    image_data = np.load(filename)
    image = Image_3d(image_data, image_meta)
    meta.add_para_xy(image, (i * 3.42, 0, 0), 15)

for i in tqdm_notebook(range(270)):
    filename = filename_prefix2 + str(i) + '.npy'
    image_data = np.load(filename)
    image = Image_3d(image_data, image_meta)
    meta.add_para_z(image, (0, 0, i * 3.42), 15)
# meta.save_h5()
# image_meta.save_h5()
from srf2.data.psf import *

psf = PSF_3d(meta, image_meta)
mat_xy = psf.generate_matrix_xy()
mat_z = psf.generate_matrix_z()
psf.save_h5()


In [ ]:

filename_prefix1 = '/home/bill52547/Workspace/SRF_new_start/run/psf_2m/psf_img_xy_'
image0 = np.zeros((201, 201))
i = 20
image0[100 + i, 100] = 1
image1 = psf2.matrix_xy * image0.flatten()
image1 = image1.reshape((201, 201))

filename = filename_prefix1 + str(i) + '.npy'
image_data = np.load(filename)
image_meta = Image_meta_3d((201,201,540), (0,0,1.71), (687.42,687.42,1846.8))
image_real = Image_3d(image_data, image_meta)
image_real = image_real.normalize()
plt.figure(figsize=(15,15))
plt.subplot(421)
plt.imshow(image_real.data[i + 90:i + 110, 90: 110, 269])
plt.colorbar()
plt.subplot(422)
plt.imshow(image1[90 + i:110 + i, 90: 110])
plt.colorbar()
plt.subplot(423)
plt.plot(range(-100, 101), image_real.data[i + 100, :, 269], 'r')
plt.subplot(424)
plt.plot(range(-100, 101), image_real.data[:, 100, 269], 'r')
plt.subplot(425)
plt.plot(range(-100, 101), image1[i + 100, :], 'b')
plt.subplot(426)
plt.plot(range(-100, 101), image1[:, 100], 'b')
import scipy.optimize as opt
def least_sq(A, y, niter = 500):
    x = np.zeros(y.shape)
    eff = A.transpose() * (A * np.ones(y.shape))
    for i in tqdm_notebook(range(niter)):
        x1 = y - A * x
        x += (A.transpose() * x1) / eff
    return x
image_solve = least_sq(psf2.matrix_xy, image_real.data[:,:, 269].ravel())
plt.subplot(427)
plt.imshow(image_solve.reshape((201,201))[i + 90:i + 110, 90: 110], vmin = 0)
plt.colorbar()


In [ ]:
image0 = np.zeros((540,))
i = 200
image0[270 + i] = 1
image1 = psf2.matrix_z * image0

filename = filename_prefix2 + str(i) + '.npy'
image_data = np.load(filename)
image_real = image_data[90:111, 90:111, :]
image_real = np.sum(image_data, axis = (0, 1))
print(image_real.shape)
plt.figure(figsize = (15, 15))
plt.subplot(211)
plt.plot(np.arange(-270 - i, 270 - i), image_real)
plt.subplot(212)
plt.plot(np.arange(-270 - i, 270 - i), image1)
